# AsterixDB

In [1]:
from asterixdb.asterixdb import AsterixConnection
from glob import glob
import json
import os
from datetime import datetime
import numpy as np

## Connect to Asterix

Establish connection to AsterixDB:

(using https://github.com/j-goldsmith/asterixdb-python)

In [2]:
con = AsterixConnection(server='http://localhost', port=19002)

Example query:

In [21]:
# response = con.query('''
#     USE TinySocial;

#     SELECT VALUE user
#     FROM GleambookUsers user
#     WHERE user.id = 8;''')

# response.results

Clear dataverse if it already exists(!) and create new

In [77]:
# response = con.query('''
#     DROP DATAVERSE FactMap IF EXISTS;
#     CREATE DATAVERSE FactMap;
#     ''')

---

## Reddit

First create subset of only 2019 submissions for experimentation:

In [3]:
rt_paths = sorted(glob("/Users/anders1991/Github/FactMap/Data/reddit/2019/*.json"))
combined = ""

for p in rt_paths:
    combined += 'localhost://' + p + ','
    
combined = combined[:-1]  # remove last ','

In [4]:
q = '''
    USE FactMap;
    
    DROP TYPE submissionTypeTemp IF EXISTS;
    CREATE TYPE submissionTypeTemp as {{
        uid: uuid
    }};
    
    DROP DATASET PostsTemp IF EXISTS;
    CREATE DATASET PostsTemp(submissionTypeTemp)
        PRIMARY KEY uid AUTOGENERATED;
                
    LOAD DATASET PostsTemp
    USING localfs (("path"="{0}"),("format"="json"));
    '''.format(combined)

response = con.query(q)

Cast to intended data format and clean up temp table:

In [5]:
response = con.query('''
    USE FactMap;

    DROP DATASET posts2019 IF EXISTS;
    
    CREATE TYPE SubmissionType as {
        id: string
    };
    CREATE DATASET posts2019(SubmissionType)
        PRIMARY KEY id;

    INSERT INTO posts2019
    SELECT 
    id,
    datetime_from_unix_time_in_secs(int(created_utc)) as created_utc,
    subreddit_id,
    subreddit,
    author,
    domain,
    int(score) as score,
    int(num_comments) as num_comments,
    title,
    url
    FROM PostsTemp p;
    
    DROP DATASET PostsTemp IF EXISTS;
    DROP TYPE SubmissionTypeTemp IF EXISTS;
    ''')

Now load all reddit submissions with direct load (note BigQuery gives `int` as `string` so have to cast):

In [6]:
rt_paths = sorted(glob("/Users/anders1991/Github/FactMap/Data/reddit/201*/*.json"))
combined = ""

for p in rt_paths:
    combined += 'localhost://' + p + ','
    
combined = combined[:-1]  # remove last ,

In [7]:
q = '''
    USE FactMap;
    
    DROP TYPE submissionTypeTemp IF EXISTS;
    CREATE TYPE submissionTypeTemp as {{
        uid: uuid
    }};
    
    DROP DATASET PostsTemp IF EXISTS;
    CREATE DATASET PostsTemp(submissionTypeTemp)
        PRIMARY KEY uid AUTOGENERATED;
                
    LOAD DATASET PostsTemp
    USING localfs (("path"="{0}"),("format"="json"));
    '''.format(combined)

response = con.query(q)

Cast to intended data format and clean up temp table:

In [8]:
response = con.query('''
    USE FactMap;

    DROP DATASET posts IF EXISTS;

    CREATE DATASET posts(SubmissionType)
        PRIMARY KEY id;

    INSERT INTO posts
    SELECT 
        id,
        datetime_from_unix_time_in_secs(int(created_utc)) as created_utc,
        subreddit_id,
        subreddit,
        author,
        domain,
        int(score) as score,
        int(num_comments) as num_comments,
        title,
        url
    FROM PostsTemp p;
    
    DROP DATASET PostsTemp IF EXISTS;
    ''')

---

## ClaimReview

Load all claims, generate uid:

In [9]:
response = con.query('''
    USE FactMap;
    
    DROP TYPE ReviewsType IF EXISTS;
    CREATE TYPE ReviewsType as {
        uid: uuid
    };
    
    DROP DATASET claims IF EXISTS;
    CREATE DATASET claims(ReviewsType)
        PRIMARY KEY uid AUTOGENERATED;
                
    LOAD DATASET claims
    USING localfs (("path"="localhost:///Users/anders1991/Github/FactMap/Data/claimreviews/claims_May-01-2019.json"),("format"="json"));
    ''')

Clean up formatting:

In [10]:
response = con.query('''
    USE FactMap;
    
    CREATE DATASET reviews(ReviewsType)
        PRIMARY KEY uid;

    INSERT INTO reviews
    SELECT 
        uid,
        reviewUrl,
        claimReviewed,
        countries,
        claimReviewed_en,
        datetime_from_unix_time_in_secs(claimDate) as claimDate,
        datetime_from_unix_time_in_secs(reviewDate) as reviewDate,
        reviewAuthor,
        reviewRating,
        claimAuthor,
        tagsRaw,
        tagsNamed,
        reviewTitle
    FROM claims c;
    
    DROP DATASET claims IF EXISTS;
    ''')    

Save subset with valid/invalid ratings:

In [11]:
import pickle

In [12]:
rated = con.query('''
    USE FactMap;
    
    SELECT r.*
    FROM reviews r
    WHERE r.reviewRating.bestRating >= r.reviewRating.ratingValue
    AND r.reviewRating.ratingValue >= r.reviewRating.worstRating
    AND r.reviewRating.bestRating > r.reviewRating.worstRating;
''').results

with open('/Users/anders1991/Github/FactMap/RNN/data/rated.pickle', 'wb') as f:
    pickle.dump(rated, f, pickle.HIGHEST_PROTOCOL)
    
len(rated)

25477

In [13]:
unrated = con.query('''
    USE FactMap;
    
    SELECT r.*
    FROM reviews r
    WHERE NOT (r.reviewRating.bestRating >= r.reviewRating.ratingValue
    AND r.reviewRating.ratingValue >= r.reviewRating.worstRating
    AND r.reviewRating.bestRating > r.reviewRating.worstRating)
    OR is_null(r.reviewRating.ratingValue)
    OR is_null(r.reviewRating.worstRating)
    OR is_null(r.reviewRating.bestRating);
''').results

with open('/Users/anders1991/Github/FactMap/RNN/data/unrated.pickle', 'wb') as f:
    pickle.dump(unrated, f, pickle.HIGHEST_PROTOCOL)

len(unrated)

34597

## Word vectors (FastText)

NOTE: See vec2json2asterix.py for more efficient pipeline and split original file into 4 using:

```
split -l 500000 crawl_300d_2M.vec crawl_300d_2M
```

Then run vec2json.py on the resulting files (in parallel), and use json2asterix.py to push them to the database (in parallel).

```
USE FactMap;

DROP DATASET fasttext IF EXISTS;
DROP TYPE EmbeddingType IF EXISTS;

CREATE TYPE EmbeddingType AS {
    word: string,
    vector: [float]
};

CREATE DATASET fasttext(EmbeddingType)
    PRIMARY KEY word;

INSERT INTO fasttext
    ([
        {"word": "hello", "vector": [0.0, 0.1]}
    ]);

SELECT *
FROM fasttext;
```

Using Facebook Research's 300-dimensional vectors for 2M words trained on Common Crawl:
https://fasttext.cc/docs/en/english-vectors.html

Importantly, these were not trained on Wikipedia but on socially 'flawed' words (like our own data)!

In [3]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [4]:
fname = "/Users/anders1991/Github/FactMap/data/fasttext/crawl-300d-2M.vec"

vectors = load_vectors(fname) 

Initialize dataset:

In [5]:
response = con.query('''
    USE FactMap;

    DROP DATASET fasttext IF EXISTS;
    DROP TYPE EmbeddingType IF EXISTS;

    CREATE TYPE EmbeddingType AS {
        word: string,
        vector: [float]
    };

    CREATE DATASET fasttext(EmbeddingType)
        PRIMARY KEY word;
    ''')

Insert word embeddings into dataset:

In [3]:
failed = []

for w, v in vectors.items():
    try:
        word = w
        vec = *v,
        response = con.query('''
            USE FactMap;
            
            INSERT INTO fasttext
            ([{{"word":"{0}", "vector":{1}}}])
        '''.format(word, list(vec)))
    except:
        failed.append(w)

# print("Failed word inserts:", len(failed))

---

## Joining

### Hard join (claims)

Join on full source URL.

In [14]:
response = con.query('''
    USE FactMap;

    SET `compiler.joinmemory` "128MB";

    DROP DATASET urljoin IF EXISTS;
    DROP TYPE PostReviewType IF EXISTS;

    CREATE TYPE PostReviewType as {
        r: ReviewsType,
        p: SubmissionType
    };

    CREATE DATASET urljoin(PostReviewType)
        PRIMARY KEY r.uid, p.id;

    INSERT INTO urljoin
    SELECT *
    FROM posts p, reviews r
    WHERE r.claimAuthor.claimURL = p.url;
    ''')

In [15]:
response = con.query('''
    USE FactMap;
    
    SELECT u.*
    FROM urljoin u;
    ''')

print('Number of matches:', len(response.results))

Number of matches: 14325


So how many unique claimreviews are represented?

In [16]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM urljoin u
    LIMIT 1;
    ''')
print('Number of unique claims:', response.results[0]['c'])

Number of unique claims: 1652


1652 unique claims are posted across the 14325 reddit submissions.

And how many of the ratings are numerically valid/invalid?

In [17]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM urljoin u
    WHERE 
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, valid numerical ratings:', response.results[0]['c'])

Number of unique, valid numerical ratings: 923


In [18]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM urljoin u
    WHERE 
    NOT
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, invalid numerical ratings:', response.results[0]['c'])

Number of unique, invalid numerical ratings: 723


### Hard join (reviews)

In [20]:
response = con.query('''
    USE FactMap;

    SET `compiler.joinmemory` "128MB";

    DROP DATASET facturljoin IF EXISTS;
    
    CREATE DATASET facturljoin(PostReviewType)
        PRIMARY KEY r.uid, p.id;

    INSERT INTO facturljoin
    SELECT *
    FROM posts p, reviews r
    WHERE r.reviewUrl = p.url;
    ''')

In [21]:
response = con.query('''
    USE FactMap;
    
    SELECT u.*
    FROM facturljoin u;
    ''')

print('Number of matches:', len(response.results))

Number of matches: 19224


In [3]:
response = con.query('''
    USE FactMap;

    SELECT COUNT(DISTINCT r.uid) unique_claims
    FROM facturljoin f;
    ''')

print('Number of unique claims', response.results[0]['unique_claims'])

Number of unique claims 8636


So there's actually more corrected news posted to reddit than fake news!

In [24]:
response = con.query('''
    USE FactMap;

    SELECT COUNT(r.uid) as c, g
    FROM facturljoin f
    GROUP BY r.uid
    GROUP AS g
    ORDER BY c desc
    LIMIT 5;
    ''').results

In [25]:
for i in response:
    print("Number of cross-posts:", i['c'])
    
    subs = [p['f']['p']['subreddit'] for p in i['g']]
    print("Subreddits:\n", subs)
    print()

Number of cross-posts: 36
Subreddits:
 ['DieOff', 'worldpolitics', 'u_the_foreign_code', 'TrumptASStic', 'CringeAnarchy', 'gogopgo', 'u_the_economy_club', 'politics', 'TrumpTellsTales', 'redacted', 'LyingTrump', 'Laserlike_Hodgepodge', 'topofreddit', 'worldnews', 'democrats', 'POLITIC', 'u_talesofafallenforest', 'POLITIC', 'POLITIC', 'POLITIC', 'chomsky', 'LeftCentral', 'politics', 'uspolitics', 'worldpolitics', 'DieOff', 'redacted', 'moderatepolitics', 'Libertarian', 'Impeach_Trump', 'Fuck45', 'esist', 'POLITIC', 'EnoughTrumpSpam', 'NoShitSherlock', 'POLITIC']

Number of cross-posts: 36
Subreddits:
 ['DieOff', 'CringeAnarchy', 'worldpolitics', 'u_the_economy_club', 'TrumptASStic', 'TrumpTellsTales', 'gogopgo', 'u_the_foreign_code', 'politics', 'redacted', 'politics', 'worldpolitics', 'DieOff', 'redacted', 'Libertarian', 'uspolitics', 'LyingTrump', 'moderatepolitics', 'topofreddit', 'worldnews', 'esist', 'democrats', 'POLITIC', 'u_talesofafallenforest', 'POLITIC', 'LeftCentral', 'Impea

On further inspection, there's no need for further fuzzy matching on the review matches.

### Hard join w/ fuzzy join on Twitter/Wikipedia (claims)

More refined join, using hard URL join only except for wikipedia and twitter posts, since these URLs to these sites are not claim specific and may refer to multiple tweets/sections.

Additional requirements for Twitter/Wikipedia posts include:
- The post domain must contain either `wikipedia` or `twitter`.
- Minimum caption length is 15 characters (cannot meaningfully distinguish claims below this threshold).
- Similar-length claim and post titles must have at least 20% word tokens in common.
    - Similar-length is defined as the difference in length between the two titles does not exceed 20% of the shortest title.
- Different-length claim and post titles must be contained within one or the other, changing at most 50% of the characters to generate a perfect match to the containing title.
    - Different-length is defined as the difference in length between the two titles must be at least 20% of the shortest title.

Create table for fuzzy URL join:

In [22]:
response = con.query('''
    USE FactMap;

    SET `compiler.joinmemory` "128MB";

    DROP DATASET fuzzyurljoin IF EXISTS;
    
    CREATE DATASET fuzzyurljoin(PostReviewType)
        PRIMARY KEY r.uid, p.id;
    ''')

Get URL matches with same-length review and post titles (Jaccard similarity > 20%):

In [23]:
response = con.query('''
        USE FactMap;
        
        INSERT INTO fuzzyurljoin
        SELECT u.*
        FROM urljoin u
        WHERE
            (similarity_jaccard(word_tokens(lower(p.title)), word_tokens(lower(r.claimReviewed))) > 0.20
            OR similarity_jaccard(word_tokens(lower(p.title)), word_tokens(lower(r.claimReviewed_en))) > 0.20)
            AND (abs(length(r.claimReviewed) - length(p.title)) <= 
                (array_min([length(r.claimReviewed), length(p.title)])) * 0.2)
            AND (array_min([length(r.claimReviewed), length(p.title)]) > 15)
            AND (contains(p.domain, "wikipedia") OR contains(p.domain, "twitter"));
        ''')

= 227 matches added.

Get URL matches with different-length review and post titles (edit distance > 0.5 * min(review or post title length)):

In [24]:
response = con.query('''
    USE FactMap;
    
    INSERT INTO fuzzyurljoin
    SELECT u.*
    FROM urljoin u
    WHERE
            (
                (edit_distance_contains(lower(p.title), lower(r.claimReviewed), length(r.claimReviewed) * 0.5)[0] 
                    OR edit_distance_contains(lower(r.claimReviewed), lower(p.title), length(p.title) * 0.5)[0])
                OR
                (edit_distance_contains(lower(p.title), lower(r.claimReviewed_en), length(r.claimReviewed_en) * 0.5)[0] 
                    OR edit_distance_contains(lower(r.claimReviewed_en), lower(p.title), length(p.title) * 0.5)[0])
            )
            AND (abs(length(r.claimReviewed) - length(p.title)) > 
                (array_min([length(r.claimReviewed), length(p.title)])) * 0.2)
            AND (array_min([length(r.claimReviewed), length(p.title)]) > 15)
            AND (contains(p.domain, "wikipedia") OR contains(p.domain, "twitter"));
    ''')

= 1166 matches added.

In [25]:
response = con.query('''
    USE FactMap;
    
    INSERT INTO fuzzyurljoin
    SELECT u.*
    FROM urljoin u
    WHERE NOT (contains(p.domain, "wikipedia") OR contains(p.domain, "twitter"));
    ''')

= 6809 matches added.

In [26]:
response = con.query('''
    USE FactMap;
    
    SELECT COUNT(*) as total
    FROM fuzzyurljoin u;
    ''')

print('Number of matches:', response.results[0]['total'])

Number of matches: 8202


From 14325 matches to 8202. A reduction of 6323 false matches.

In [27]:
response = con.query('''
    USE FactMap;

    SELECT COUNT(DISTINCT r.uid) unique_claims
    FROM fuzzyurljoin f;
    ''')

print('Number of unique claims', response.results[0]['unique_claims'])

Number of unique claims 1378


From 1652 unique claims to 1378. A reduction of 274 falsely matched claims.

In [28]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM fuzzyurljoin u
    WHERE 
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, valid numerical ratings:', response.results[0]['c'])

Number of unique, valid numerical ratings: 725


In [29]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM fuzzyurljoin u
    WHERE 
    NOT
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, invalid numerical ratings:', response.results[0]['c'])

Number of unique, invalid numerical ratings: 647


----

Helpful queries:

In [ ]:
# r = con.query('''
#     use FactMap;
#     SELECT VALUE ds FROM Metadata.`Dataset` ds WHERE DataverseName = 'FactMap';
#     SELECT VALUE ds FROM Metadata.`Dataset` ds;
#     SELECT VALUE ix FROM Metadata.`Index` ix;''')

# r.results